<a href="https://colab.research.google.com/github/tlysenko/Machine-Learning-Engineering-for-Production-MLOps-/blob/main/Week2/C2W2_Feature_Engineering_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing the libraries

In [ ]:
!pip install tfx

## Importing the libraries

In [ ]:
import tensorflow as tf

from tfx import v1 as tfx

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from google.protobuf.json_format import MessageToDict

import pandas as pd

import os
import pprint
pp = pprint.PrettyPrinter()

## Getting the data

In [ ]:
data_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00560/SeoulBikeData.csv'

data = pd.read_csv(data_url, index_col=False, skiprows=0, header = 0, encoding= 'unicode_escape')
data.columns = ['date',
 'rented-bike-count',
 'hour',
 'temperature',
 'humidity',
 'wind-speed',
 'visibility',
 'dew-point-temperature',
 'solar-radiation',
 'rainfall',
 'snowfall',
 'seasons',
 'holiday',
 'functioning-day']

data.head()

,date,rented-bike-count,hour,temperature,humidity,wind-speed,visibility,dew-point-temperature,solar-radiation,rainfall,snowfall,seasons,holiday,functioning-day
0,01/12/2017,254,0,-5.2,37,2.2,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
1,01/12/2017,204,1,-5.5,38,0.8,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
2,01/12/2017,173,2,-6.0,39,1.0,2000,-17.7,0.0,0.0,0.0,Winter,No Holiday,Yes
3,01/12/2017,107,3,-6.2,40,0.9,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
4,01/12/2017,78,4,-6.0,36,2.3,2000,-18.6,0.0,0.0,0.0,Winter,No Holiday,Yes


## Define paths

In [ ]:
# location of the pipeline metadata store
_pipeline_root = './pipeline/'
!mkdir {_pipeline_root}
!mkdir './data/'

# directory of the raw data files
_data_root = './data/seul_bike/'
!mkdir {_data_root}

# path to the raw training data
_data_filepath = os.path.join(_data_root, 'SeulBikeData.csv')

mkdir: cannot create directory ‘./pipeline/’: File exists
mkdir: cannot create directory ‘./data/’: File exists
mkdir: cannot create directory ‘./data/seul_bike/’: File exists


## Download the data

In [ ]:
data.to_csv(_data_filepath)

## Preview dataset

In [ ]:
!head {_data_filepath}

,date,rented-bike-count,hour,temperature,humidity,wind-speed,visibility,dew-point-temperature,solar-radiation,rainfall,snowfall,seasons,holiday,functioning-day
0,01/12/2017,254,0,-5.2,37,2.2,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
1,01/12/2017,204,1,-5.5,38,0.8,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
2,01/12/2017,173,2,-6.0,39,1.0,2000,-17.7,0.0,0.0,0.0,Winter,No Holiday,Yes
3,01/12/2017,107,3,-6.2,40,0.9,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
4,01/12/2017,78,4,-6.0,36,2.3,2000,-18.6,0.0,0.0,0.0,Winter,No Holiday,Yes
5,01/12/2017,100,5,-6.4,37,1.5,2000,-18.7,0.0,0.0,0.0,Winter,No Holiday,Yes
6,01/12/2017,181,6,-6.6,35,1.3,2000,-19.5,0.0,0.0,0.0,Winter,No Holiday,Yes
7,01/12/2017,460,7,-7.4,38,0.9,2000,-19.3,0.0,0.0,0.0,Winter,No Holiday,Yes
8,01/12/2017,930,8,-7.6,37,1.1,2000,-19.8,0.01,0.0,0.0,Winter,No Holiday,Yes


## Creating the interactive context

In [ ]:
# Instantiate ExampleGen with the input CSV dataset
context = InteractiveContext(pipeline_root=_pipeline_root)

## Initialize ExampleGen

In [ ]:
example_gen = tfx.components.CsvExampleGen(input_base=_data_root)

In [ ]:
# Execute the component
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 27
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 62
        type_id: 14
        uri: "./pipeline/CsvExampleGen/examples/27"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "file_format"
          value {
            string_value: "tfrecords_gzip"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:685624,xor_checksum:1642514308,sum_checksum:1642514308"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        base_type: DATASET
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

## Examine the output artifacts programmatically

In [ ]:
# get the artifact object
artifact = example_gen.outputs['examples'].get()[0]

In [ ]:
type(artifact)

tfx.types.standard_artifacts.Examples

In [ ]:
print(artifact.split_names)

["train", "eval"]


In [ ]:
print(artifact.uri)

./pipeline/CsvExampleGen/examples/27


In [ ]:
# Get the URI of the output artifact representing the training examples
train_uri = os.path.join(artifact.uri, 'Split-train')

# See the contents of the `train` folder
!ls {train_uri}

data_tfrecord-00000-of-00001.gz


In [ ]:
# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

In [ ]:
print(tfrecord_filenames)

['./pipeline/CsvExampleGen/examples/27/Split-train/data_tfrecord-00000-of-00001.gz']


In [ ]:
# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

In [ ]:
# Define a helper function to get individual examples
def get_records(dataset, num_records):
    '''Extracts records from the given dataset.
    Args:
        dataset (TFRecordDataset): dataset saved by ExampleGen
        num_records (int): number of records to preview
    '''
    
    # initialize an empty list
    records = []
    
    # Use the `take()` method to specify how many records to get
    for tfrecord in dataset.take(num_records):
        
        # Get the numpy property of the tensor
        serialized_example = tfrecord.numpy()
        
        # Initialize a `tf.train.Example()` to read the serialized data
        example = tf.train.Example()
        
        # Read the example data (output is a protocol buffer message)
        example.ParseFromString(serialized_example)
        
        # convert the protocol bufffer message to a Python dictionary
        example_dict = (MessageToDict(example))
        
        # append to the records list
        records.append(example_dict)
        
    return records

In [ ]:
# Get 3 records from the dataset
sample_records = get_records(dataset, 3)

# Print the output
pp.pprint(sample_records)

[{'features': {'feature': {'': {'int64List': {'value': ['1']}},
                           'date': {'bytesList': {'value': ['MDEvMTIvMjAxNw==']}},
                           'dew-point-temperature': {'floatList': {'value': [-17.6]}},
                           'functioning-day': {'bytesList': {'value': ['WWVz']}},
                           'holiday': {'bytesList': {'value': ['Tm8gSG9saWRheQ==']}},
                           'hour': {'int64List': {'value': ['1']}},
                           'humidity': {'int64List': {'value': ['38']}},
                           'rainfall': {'floatList': {'value': [0.0]}},
                           'rented-bike-count': {'int64List': {'value': ['204']}},
                           'seasons': {'bytesList': {'value': ['V2ludGVy']}},
                           'snowfall': {'floatList': {'value': [0.0]}},
                           'solar-radiation': {'floatList': {'value': [0.0]}},
                           'temperature': {'floatList': {'value': [-5.5]}

## StatisticsGen

In [ ]:
# Instantiate StatisticsGen with the ExampleGen ingested dataset
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])

# Execute the component
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 28
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 63
        type_id: 16
        uri: "./pipeline/StatisticsGen/statistics/28"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 16
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        base_type: STATISTICS
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [ ]:
# Show the output statistics
context.show(statistics_gen.outputs['statistics'])

## SchemaGen

In [ ]:
# Instantiate SchemaGen with the StatisticsGen ingested dataset
schema_gen = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    )

# Run the component
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 29
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 64
        type_id: 18
        uri: "./pipeline/SchemaGen/schema/29"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 18
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [ ]:
# Visualize the schema
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'',INT,required,,-
'date',BYTES,required,,-
'dew-point-temperature',FLOAT,required,,-
'functioning-day',STRING,required,,'functioning-day'
'holiday',STRING,required,,'holiday'
'hour',INT,required,,-
'humidity',INT,required,,-
'rainfall',FLOAT,required,,-
'rented-bike-count',INT,required,,-


,Values
Domain,
'functioning-day',"'No', 'Yes'"
'holiday',"'Holiday', 'No Holiday'"
'seasons',"'Autumn', 'Spring', 'Summer', 'Winter'"


## ExampleValidator

In [ ]:
# Instantiate ExampleValidator with the StatisticsGen and SchemaGen ingested data
example_validator = tfx.components.ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])

# Run the component.
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 30
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 65
        type_id: 20
        uri: "./pipeline/ExampleValidator/anomalies/30"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 20
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [ ]:
# Visualize the results
context.show(example_validator.outputs['anomalies'])

## Transform

In [ ]:
# Set the constants module filename
_census_constants_module_file = 'census_constants.py'

In [ ]:
%%writefile {_census_constants_module_file}

# Features with string data types that will be converted to indices
CATEGORICAL_FEATURE_KEYS = [
    'seasons', 'holiday', 'functioning-day']

# Numerical features that are marked as continuous
NUMERIC_FEATURE_KEYS = ['temperature', 'wind-speed','dew-point-temperature',
                       'solar-radiation', 'rainfall', 'snowfall', 'hour','humidity','visibility']


# Feature that can be grouped into buckets
BUCKET_FEATURE_KEYS = ['humidity']

# Number of buckets used by tf.transform for encoding each bucket feature.
FEATURE_BUCKET_COUNT = {'humidity': 8}

# Feature that the model will predict
LABEL_KEY = 'rented-bike-count'

# Utility function for renaming the feature
def transformed_name(key):
    return key + '_xf'

Overwriting census_constants.py


In [ ]:
# Set the transform module filename
_census_transform_module_file = 'census_transform.py'

In [ ]:
%%writefile {_census_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

import census_constants

# Unpack the contents of the constants module
_NUMERIC_FEATURE_KEYS = census_constants.NUMERIC_FEATURE_KEYS
_CATEGORICAL_FEATURE_KEYS = census_constants.CATEGORICAL_FEATURE_KEYS
_BUCKET_FEATURE_KEYS = census_constants.BUCKET_FEATURE_KEYS
_FEATURE_BUCKET_COUNT = census_constants.FEATURE_BUCKET_COUNT

_LABEL_KEY = census_constants.LABEL_KEY
_transformed_name = census_constants.transformed_name


# Define the transformations
def preprocessing_fn(inputs):
    """tf.transform's callback function for preprocessing inputs.
    Args:
        inputs: map from feature keys to raw not-yet-transformed features.
    Returns:
        Map from string feature key to transformed feature operations.
    """
    outputs = {}

    # Scale these features to the range [0,1]
    for key in _NUMERIC_FEATURE_KEYS:
        outputs[_transformed_name(key)] = tft.scale_to_0_1(
            inputs[key])
    
    # Bucketize these features
    for key in _BUCKET_FEATURE_KEYS:
        outputs[_transformed_name(key)] = tft.bucketize(
            inputs[key], _FEATURE_BUCKET_COUNT[key])
      

    # Convert strings to indices in a vocabulary
    for key in _CATEGORICAL_FEATURE_KEYS:
        outputs[_transformed_name(key)] = tft.compute_and_apply_vocabulary(inputs[key])

    # Convert the label strings to an index
    outputs[_transformed_name(_LABEL_KEY)] = tft.compute_and_apply_vocabulary(inputs[_LABEL_KEY])

    return outputs

Overwriting census_transform.py


In [ ]:
# Ignore TF warning messages
tf.get_logger().setLevel('ERROR')

# Instantiate the Transform component
transform = tfx.components.Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(_census_transform_module_file))

# Run the component
context.run(transform)

ExecutionResult(
    component_id: Transform
    execution_id: 36
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 106
        type_id: 22
        uri: "./pipeline/Transform/transform_graph/36"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 22
        name: "TransformGraph"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 107
        type_id: 14
        uri: "./pipeline/Transform/transformed_examples/36"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        base_type: DATASET
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 108
        type_id: 23
        uri: "./pipeline/Transform/updated_analyzer_cache/36"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 23
        name: "TransformCache"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        pre_transform_schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 109
        type_id: 18
        uri: "./pipeline/Transform/pre_transform_schema/36"
        custom_properties {
          key: "name"
          value {
            string_value: "pre_transform_schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
 

In [ ]:
# Get the uri of the transform graph
transform_graph_uri = transform.outputs['transform_graph'].get()[0].uri

# List the subdirectories under the uri
os.listdir(transform_graph_uri)

['transformed_metadata', 'metadata', 'transform_fn']

In [ ]:
# Get the URI of the output artifact representing the transformed examples
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
transformed_dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

In [ ]:
# Get 3 records from the dataset
sample_records_xf = get_records(transformed_dataset, 3)

# Print the output
pp.pprint(sample_records_xf)

[{'features': {'feature': {'dew-point-temperature_xf': {'floatList': {'value': [0.22648083]}},
                           'functioning-day_xf': {'int64List': {'value': ['0']}},
                           'holiday_xf': {'int64List': {'value': ['0']}},
                           'hour_xf': {'floatList': {'value': [0.04347826]}},
                           'humidity_xf': {'int64List': {'value': ['1']}},
                           'rainfall_xf': {'floatList': {'value': [0.0]}},
                           'rented-bike-count_xf': {'int64List': {'value': ['236']}},
                           'seasons_xf': {'int64List': {'value': ['2']}},
                           'snowfall_xf': {'floatList': {'value': [0.0]}},
                           'solar-radiation_xf': {'floatList': {'value': [0.0]}},
                           'temperature_xf': {'floatList': {'value': [0.21541154]}},
                           'visibility_xf': {'floatList': {'value': [1.0]}},
                           'wind-speed_xf'